# Book Recommendation System

- Content-based collaborative filtering can utilize various attributes such as the title, author, publisher, and category as key features for recommendation. In this approach, recommendations are generated based on the similarities between the items, determined through matching these attributes. For example, a book sharing a similar author or category with a user's previously liked books may be recommended.

## Tentang Dataset
Dataset ini mencakup 278,858 pengguna yang memberikan 1,149,780 penilaian (baik eksplisit maupun implisit) terhadap 271,379 buku. Berikut adalah deskripsi fitur yang tersedia:

- **user_id**: Identifikasi unik pengguna
- **location**: Alamat atau lokasi pengguna
- **age**: Usia pengguna
- **isbn**: ISBN (International Standard Book Number) buku
- **rating**: Penilaian yang diberikan untuk buku
- **book_title**: Nama atau judul buku
- **book_author**: Penulis buku
- **year_of_publication**: Tahun buku diterbitkan
- **publisher**: Penerbit buku
- **img_s**: Gambar sampul buku ukuran kecil
- **img_m**: Gambar sampul buku ukuran sedang
- **img_l**: Gambar sampul buku ukuran besar
- **Summary**: Sinopsis atau ringkasan buku
- **Language**: Bahasa buku
- **Category**: Kategori atau genre buku
- **city**: Kota pengguna
- **state**: Negara bagian pengguna
- **country**: Negara tempat pengguna berada

## Libraries

In [2]:
# Install the opendatasets library
%pip install opendatasets


Perintah di atas digunakan untuk menginstal library `opendatasets`, yang memungkinkan kita untuk mengunduh dataset dari berbagai sumber secara langsung ke lingkungan kerja. Pastikan lingkungan Python Anda sudah mendukung penggunaan `%pip` sebagai perintah magic. Jika tidak, Anda bisa menjalankan perintah yang sama di terminal atau command prompt tanpa simbol `%`.

In [3]:
# Mengimpor berbagai pustaka yang diperlukan
import os
import re
import nltk
import requests
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import opendatasets as od

# Mengimpor stopwords dari NLTK
from nltk.corpus import stopwords
nltk.download("stopwords")  # Mengunduh daftar stopwords dari NLTK

# Mengimpor CountVectorizer untuk ekstraksi fitur teks
from sklearn.feature_extraction.text import CountVectorizer
# Mengimpor cosine_similarity untuk menghitung kesamaan antar vektor
from sklearn.metrics.pairwise import cosine_similarity

# Mengimpor pustaka PIL untuk manipulasi gambar
from PIL import Image


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


- Setiap pustaka yang diimpor memiliki fungsinya masing-masing.
- `os` dan `re` digunakan untuk operasi sistem dan ekspresi reguler.
- `nltk` adalah pustaka untuk pengolahan bahasa alami.
- `requests` dan `warnings` digunakan untuk melakukan permintaan HTTP dan mengelola peringatan, masing-masing.
- `pandas` dan `numpy` adalah pustaka untuk manipulasi data dan komputasi numerik.
- `matplotlib.pyplot` digunakan untuk visualisasi data.
- `opendatasets` memudahkan pengunduhan dataset dari sumber online.
- `CountVectorizer` dan `cosine_similarity` adalah alat dari Scikit-Learn untuk memproses teks dan menghitung kesamaan antar teks.
- `PIL` membantu dalam pemrosesan gambar, seperti membuka dan mengedit file gambar.

## Load and Check Dataset

In [4]:
# Mengunduh dataset dari Kaggle menggunakan pustaka opendatasets
od.download('https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset')


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: khamdanfakhryza
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset


100%|██████████| 76.1M/76.1M [00:00<00:00, 99.8MB/s]


- Perintah ini menggunakan fungsi `download` dari pustaka `opendatasets` untuk mengambil dataset yang berada di tautan Kaggle.
- Pastikan Anda telah menginstal pustaka `opendatasets` sebelumnya dan memiliki akses ke dataset di Kaggle.
- Setelah proses pengunduhan selesai, dataset akan disimpan dalam direktori kerja Anda saat ini, memudahkan akses dan analisis lebih lanjut.

In [6]:
# Membaca file CSV yang berisi data buku dan menyimpannya dalam DataFrame
books = pd.read_csv('/content/bookcrossing-dataset/Books Data with Category Language and Summary/Preprocessed_data.csv')
# Menampilkan dua baris pertama dari DataFrame untuk melihat struktur data
books.head(2)


,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada


- Kode ini menggunakan fungsi `pd.read_csv` dari pustaka `pandas` untuk memuat file CSV yang berisi informasi buku.
- Data ini disimpan dalam variabel `books` sebagai DataFrame, yang memudahkan manipulasi dan analisis data.
- Fungsi `head(2)` digunakan untuk menampilkan dua baris pertama dari DataFrame, memberikan gambaran awal tentang data yang dimuat, termasuk kolom dan tipe datanya.

In [7]:
# Menampilkan informasi mengenai DataFrame books, termasuk tipe data dan jumlah entri
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031175 entries, 0 to 1031174
Data columns (total 19 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Unnamed: 0           1031175 non-null  int64  
 1   user_id              1031175 non-null  int64  
 2   location             1031175 non-null  object 
 3   age                  1031175 non-null  float64
 4   isbn                 1031175 non-null  object 
 5   rating               1031175 non-null  int64  
 6   book_title           1031175 non-null  object 
 7   book_author          1031174 non-null  object 
 8   year_of_publication  1031175 non-null  float64
 9   publisher            1031175 non-null  object 
 10  img_s                1031175 non-null  object 
 11  img_m                1031175 non-null  object 
 12  img_l                1031175 non-null  object 
 13  Summary              1031175 non-null  object 
 14  Language             1031175 non-null  object 
 15

- Kode ini menggunakan metode `info()` dari objek DataFrame `books` untuk memberikan ringkasan informasi tentang dataset.
- Informasi yang ditampilkan mencakup jumlah total entri, nama kolom, tipe data setiap kolom, dan jumlah nilai non-null, yang sangat berguna untuk memahami struktur dan kualitas data sebelum analisis lebih lanjut.

In [8]:
# Menampilkan semua nilai unik dari kolom rating dan mengurutkannya
print(sorted(books.rating.unique()))
print()  # Menambahkan baris kosong untuk pemisahan output

# Menampilkan jumlah setiap nilai rating dalam dataset
print(books.rating.value_counts())


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

rating
0     647323
8      91806
10     71227
7      66404
9      60780
5      45355
6      31689
4       7617
3       5118
2       2375
1       1481
Name: count, dtype: int64


- Kode pertama menggunakan `unique()` untuk mengambil semua nilai yang berbeda dari kolom `rating` dan kemudian mengurutkannya dengan `sorted()`, sehingga kita dapat melihat rentang penilaian yang ada.
- Baris kedua menambahkan spasi kosong untuk memisahkan output, membuatnya lebih mudah dibaca.
- Kode kedua memanfaatkan `value_counts()` untuk menghitung frekuensi kemunculan setiap nilai rating dalam dataset, yang memberikan wawasan tentang distribusi penilaian buku oleh pengguna.

In [9]:
# Menghitung jumlah nilai yang hilang (null) dalam setiap kolom DataFrame books
books.isnull().sum()


,0
Unnamed: 0,0
user_id,0
location,0
age,0
isbn,0
rating,0
book_title,0
book_author,1
year_of_publication,0
publisher,0


**Catatan (Bahasa Indonesia):**
- Kode ini menggunakan metode `isnull()` untuk mendeteksi nilai null dalam DataFrame `books`, kemudian diikuti oleh `sum()` untuk menjumlahkan jumlah nilai null di setiap kolom.
- Output yang dihasilkan akan menunjukkan kolom mana yang memiliki data hilang dan berapa banyak nilai yang hilang, sehingga kita dapat mengambil langkah-langkah perbaikan yang diperlukan sebelum melakukan analisis lebih lanjut.

In [10]:
# Menampilkan semua kategori unik dari kolom Category dalam DataFrame books
print(books.Category.unique())
print()  # Menambahkan baris kosong untuk memisahkan output

# Menampilkan daftar kategori yang ada berdasarkan frekuensi penilaian
print(books.Category.value_counts().index)


["['Social Science']" "['Actresses']" "['1940-1949']" ...
 "['Microsoft Windows NT.']" "['Merchants']" "['Alternative histories']"]

Index(['9', '['Fiction']', '['Juvenile Fiction']',
       '['Biography & Autobiography']', '['Humor']', '['History']',
       '['Religion']', '['Juvenile Nonfiction']', '['Social Science']',
       '['Body, Mind & Spirit']',
       ...
       '['Human-alien encounters.']', '['Adel']',
       '['Brobdingnag (Imaginary place)']', '['Devotional literature.']',
       '['Tourism']', '['Angel (Fictitious character : Whedon)']', '['Face']',
       '['Church renewal']', '['Supermarkets']', '['Alternative histories']'],
      dtype='object', name='Category', length=6448)


- Kode pertama menggunakan `unique()` untuk mendapatkan semua kategori yang berbeda dari kolom `Category`, memberikan pandangan tentang berbagai genre atau kategori buku yang tersedia dalam dataset.
- Baris kedua menambahkan spasi kosong untuk memisahkan output, membuat hasilnya lebih teratur dan mudah dibaca.
- Kode kedua memanfaatkan `value_counts()` untuk menghitung frekuensi kemunculan setiap kategori dan kemudian menampilkan indeksnya, yang memberi tahu kita kategori mana yang paling banyak ditemukan dalam data.

In [11]:
# Menggunakan kolom Publisher sebagai fitur
# Menampilkan jumlah kemunculan setiap penerbit dalam DataFrame books
books.publisher.value_counts()


,count
publisher,
Ballantine Books,34724
Pocket,31989
Berkley Publishing Group,28614
Warner Books,25506
Harlequin,25029
...,...
"Langley Press, Incorporated",1
Division of Archives and Hist Tural Resources,1
Terra Nova Press,1


- Kode ini menghitung dan menampilkan jumlah setiap penerbit (publisher) yang terdapat dalam kolom `publisher` dari DataFrame `books`.
- Fungsi `value_counts()` digunakan untuk memberikan informasi tentang berapa kali setiap penerbit muncul dalam dataset, yang berguna untuk menganalisis distribusi buku berdasarkan penerbit.
- Ini dapat membantu dalam memahami kontribusi penerbit tertentu terhadap keseluruhan dataset dan mendeteksi pola dalam penerbitan buku.

## Preprocessing

In [12]:
# Membuat salinan dari DataFrame books untuk diproses lebih lanjut
df = books.copy()

# Menghapus baris yang memiliki nilai null di mana saja
df.dropna(inplace=True, how='any', axis=0)

# Mengatur ulang indeks DataFrame setelah penghapusan
df.reset_index(drop=True, inplace=True)

# Menghapus kolom yang tidak akan digunakan dalam analisis
df.drop(columns=['Unnamed: 0', 'location', 'isbn',
                 'img_s', 'img_m', 'img_l', 'city', 'age',
                 'state', 'Language', 'country',
                 'year_of_publication', 'Summary'], axis=1, inplace=True)  # Kolom yang dihapus tidak akan digunakan

# Menghapus baris yang memiliki kategori '9'
df.drop(index=df[df.Category == '9'].index, inplace=True)

# Menghapus baris yang memiliki rating 0
df.drop(index=df[df.rating == 0].index, inplace=True)

# Menghapus karakter non-alfanumerik dari kolom Category
df.Category = df.Category.apply(lambda x: re.sub('[\W_]+', ' ', x).strip())

# Menampilkan dua baris pertama dari DataFrame yang telah diproses
df.head()


,user_id,rating,book_title,book_author,publisher,Category
1,8,5,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,Actresses
4,67544,8,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,Actresses
7,123629,9,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,Actresses
9,200273,8,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,Actresses
10,210926,9,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,Actresses


- Kode ini dimulai dengan membuat salinan dari DataFrame `books` untuk mencegah perubahan pada data asli.
- Fungsi `dropna()` digunakan untuk menghapus baris yang memiliki nilai null di salah satu kolom, memastikan bahwa analisis hanya dilakukan pada data yang lengkap.
- Metode `reset_index()` membantu mengatur ulang indeks setelah penghapusan, sehingga tidak ada celah pada indeks.
- Beberapa kolom yang tidak diperlukan dihapus dari DataFrame dengan `drop()`, membuat dataset lebih bersih dan fokus pada informasi yang relevan.
- Kode ini juga menghapus baris dengan kategori '9' dan rating 0 untuk menyaring data yang tidak diinginkan.
- Fungsi `apply()` bersama dengan `re.sub()` digunakan untuk membersihkan teks dalam kolom `Category`, menghapus karakter yang tidak diinginkan.
- Akhirnya, `head()` digunakan untuk menampilkan beberapa baris pertama dari DataFrame yang telah diproses, memberikan gambaran tentang data yang telah dibersihkan.

In [13]:
# Menampilkan informasi mengenai DataFrame df, termasuk tipe data dan jumlah entri yang ada
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 217314 entries, 1 to 982276
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      217314 non-null  int64 
 1   rating       217314 non-null  int64 
 2   book_title   217314 non-null  object
 3   book_author  217314 non-null  object
 4   publisher    217314 non-null  object
 5   Category     217314 non-null  object
dtypes: int64(2), object(4)
memory usage: 11.6+ MB


- Kode ini menggunakan metode `info()` pada DataFrame `df` untuk memberikan ringkasan informasi tentang dataset setelah proses pembersihan.
- Informasi yang ditampilkan mencakup jumlah total entri, nama kolom, tipe data untuk setiap kolom, serta jumlah nilai non-null. Ini membantu untuk memahami struktur data dan memeriksa apakah proses pembersihan berhasil.

In [14]:
# Menghitung jumlah nilai yang hilang (null) dalam setiap kolom di DataFrame df
df.isnull().sum()

,0
user_id,0
rating,0
book_title,0
book_author,0
publisher,0
Category,0


- Kode ini menggunakan metode `isnull()` untuk mendeteksi nilai yang hilang dalam DataFrame `df`, diikuti dengan `sum()` untuk menghitung total nilai null di setiap kolom.
- Hasil yang diperoleh akan menunjukkan kolom mana yang memiliki data hilang dan berapa banyak nilai yang hilang, memberikan gambaran mengenai kualitas data setelah proses pembersihan. Ini penting untuk memastikan dataset siap digunakan untuk analisis lebih lanjut.

In [15]:
# Menghitung dan menampilkan 25 kategori teratas berdasarkan jumlah kemunculan
i = 1
for idx, name in enumerate(df['Category'].value_counts().index.tolist()):
    if i == 25:
        break  # Hentikan loop setelah 25 kategori ditampilkan
    print(i)  # Menampilkan nomor urut kategori
    print('Nama :', name)  # Menampilkan nama kategori
    print('Jumlah :', df['Category'].value_counts()[idx])  # Menampilkan jumlah kemunculan kategori
    print('---' * 8)  # Menambahkan pemisah antar kategori
    i += 1  # Menambah penghitung kategori


1
Nama : Fiction
Jumlah : 127055
------------------------
2
Nama : Juvenile Fiction
Jumlah : 14181
------------------------
3
Nama : Biography Autobiography
Jumlah : 8876
------------------------
4
Nama : Humor
Jumlah : 3721
------------------------
5
Nama : History
Jumlah : 3121
------------------------
6
Nama : Religion
Jumlah : 2843
------------------------
7
Nama : Body Mind Spirit
Jumlah : 1999
------------------------
8
Nama : Juvenile Nonfiction
Jumlah : 1955
------------------------
9
Nama : Social Science
Jumlah : 1937
------------------------
10
Nama : Business Economics


<ipython-input-15-536e34b84eb5>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('Jumlah :', df['Category'].value_counts()[idx])  # Menampilkan jumlah kemunculan kategori
<ipython-input-15-536e34b84eb5>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('Jumlah :', df['Category'].value_counts()[idx])  # Menampilkan jumlah kemunculan kategori
<ipython-input-15-536e34b84eb5>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p

Jumlah : 1801
------------------------
11
Nama : Family Relationships
Jumlah : 1671
------------------------
12
Nama : Self Help
Jumlah : 1644
------------------------
13
Nama : Health Fitness
Jumlah : 1514
------------------------
14
Nama : Cooking
Jumlah : 1325
------------------------
15
Nama : Travel
Jumlah : 1161
------------------------
16
Nama : Poetry
Jumlah : 985
------------------------
17
Nama : True Crime
Jumlah : 965
------------------------
18
Nama : Psychology
Jumlah : 949
------------------------
19
Nama : Science
Jumlah : 933
------------------------
20
Nama : Computers
Jumlah : 894
------------------------
21
Nama : Literary Criticism
Jumlah : 877
------------------------
22
Nama : Drama
Jumlah : 839
------------------------
23
Nama : Political Science
Jumlah : 814
------------------------
24
Nama : Philosophy
Jumlah : 772
------------------------


<ipython-input-15-536e34b84eb5>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('Jumlah :', df['Category'].value_counts()[idx])  # Menampilkan jumlah kemunculan kategori
<ipython-input-15-536e34b84eb5>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('Jumlah :', df['Category'].value_counts()[idx])  # Menampilkan jumlah kemunculan kategori
<ipython-input-15-536e34b84eb5>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p

- Kode ini digunakan untuk menghitung dan menampilkan 25 kategori buku teratas berdasarkan frekuensi kemunculan di dalam DataFrame `df`.
- Loop `for` digunakan untuk mengiterasi setiap kategori dan `enumerate()` memberikan indeks serta nama kategori.
- Jika penghitung `i` mencapai 25, loop akan dihentikan dengan `break`, sehingga hanya 25 kategori pertama yang ditampilkan.
- Dalam setiap iterasi, nomor urut kategori, nama kategori, dan jumlah kemunculannya ditampilkan, memberikan wawasan tentang distribusi kategori dalam dataset.
- Pemisah `print('---' * 8)` ditambahkan untuk membuat hasil output lebih teratur dan mudah dibaca.

In [16]:
# Membuat daftar kategori berdasarkan jumlah kemunculan dan menyimpannya dalam cat_list
cat_list = df.Category.value_counts().index.tolist()

# Menampilkan kategori dari indeks 5 hingga 19 (15 kategori) dalam cat_list
print(cat_list[5:20])


['Religion', 'Body Mind Spirit', 'Juvenile Nonfiction', 'Social Science', 'Business Economics', 'Family Relationships', 'Self Help', 'Health Fitness', 'Cooking', 'Travel', 'Poetry', 'True Crime', 'Psychology', 'Science', 'Computers']


- Kode ini pertama-tama membuat daftar `cat_list` yang berisi nama-nama kategori buku berdasarkan frekuensi kemunculan dari kolom `Category` di DataFrame `df`.
- `value_counts().index.tolist()` digunakan untuk mendapatkan urutan kategori yang sudah diurutkan berdasarkan jumlah kemunculannya.
- Kemudian, bagian dari daftar kategori yang ingin ditampilkan adalah dari indeks 5 hingga 19, yang berarti menampilkan 15 kategori berikutnya setelah kategori pertama yang ditampilkan. Ini berguna untuk melihat lebih banyak kategori yang mungkin tidak terlalu umum tetapi masih ada dalam dataset.

In [17]:
# Memfilter DataFrame df untuk hanya menyertakan kategori yang terdapat dalam cat_list dari indeks 5 hingga 19
df_fil = df[df.Category.isin(cat_list[5:20])]

# Menampilkan informasi mengenai DataFrame df_fil, termasuk tipe data dan jumlah entri
df_fil.info()


<class 'pandas.core.frame.DataFrame'>
Index: 22576 entries, 694 to 982240
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      22576 non-null  int64 
 1   rating       22576 non-null  int64 
 2   book_title   22576 non-null  object
 3   book_author  22576 non-null  object
 4   publisher    22576 non-null  object
 5   Category     22576 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


- Kode ini digunakan untuk memfilter DataFrame `df` sehingga hanya menyertakan baris yang kategori bukunya ada dalam daftar `cat_list` dari indeks 5 hingga 19.
- Fungsi `isin()` membantu untuk melakukan pemfilteran berdasarkan kondisi yang ditentukan, yaitu kategori yang ada dalam rentang tersebut.
- Hasil pemfilteran disimpan dalam DataFrame baru bernama `df_fil`.
- Kemudian, metode `info()` digunakan untuk menampilkan informasi mengenai struktur dan isi dari DataFrame yang sudah difilter, seperti jumlah entri, tipe data untuk setiap kolom, dan jumlah nilai non-null. Ini membantu memastikan bahwa filter yang diterapkan menghasilkan dataset yang diinginkan.

In [18]:
# Menghitung jumlah kategori unik yang ada di dalam DataFrame df_fil
df_fil.Category.nunique()


15

- Kode ini digunakan untuk menghitung dan mengembalikan jumlah kategori unik yang terdapat dalam kolom `Category` pada DataFrame `df_fil`.
- Fungsi `nunique()` akan memberikan jumlah nilai unik yang berbeda dalam kolom tersebut, sehingga membantu untuk memahami keragaman kategori yang ada setelah proses pemfilteran.
- Hasil dari perhitungan ini dapat memberikan wawasan mengenai seberapa banyak kategori yang tersisa dan apakah cukup representatif untuk analisis lebih lanjut.

In [19]:
# Membuat salinan dari DataFrame df_fil dan menyimpannya dalam variabel prep
prep = df_fil.copy()

# Mengurutkan DataFrame prep berdasarkan kolom 'book_title'
prep.sort_values('book_title', inplace=True)


- Kode ini pertama-tama membuat salinan dari DataFrame `df_fil` dan menyimpannya dalam variabel baru bernama `prep`. Ini berguna untuk menjaga data asli tetap utuh saat melakukan modifikasi.
- Kemudian, metode `sort_values()` digunakan untuk mengurutkan baris dalam DataFrame `prep` berdasarkan kolom `book_title`.
- Penambahan argumen `inplace=True` memastikan bahwa pengurutan dilakukan langsung pada DataFrame `prep`, tanpa perlu mengembalikannya ke variabel baru. Proses ini membantu dalam mempersiapkan data untuk analisis atau visualisasi lebih lanjut berdasarkan urutan judul buku.

In [20]:
# Menghapus duplikat berdasarkan kolom 'book_title' dari DataFrame prep
prep = prep.drop_duplicates('book_title')

# Menampilkan informasi mengenai DataFrame prep, termasuk tipe data dan jumlah entri
prep.info()

print()  # Menambahkan baris kosong untuk pemisah

# Menampilkan 4 baris pertama dari DataFrame prep
prep.head(4)


<class 'pandas.core.frame.DataFrame'>
Index: 13048 entries, 958962 to 788540
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      13048 non-null  int64 
 1   rating       13048 non-null  int64 
 2   book_title   13048 non-null  object
 3   book_author  13048 non-null  object
 4   publisher    13048 non-null  object
 5   Category     13048 non-null  object
dtypes: int64(2), object(4)
memory usage: 713.6+ KB



,user_id,rating,book_title,book_author,publisher,Category
958962,237883,9,Microsoft Application Architecture For Micros...,Microsoft Corporation Staff,Microsoft Press,Computers
862827,131193,8,$30 Film School,Michael W. Dean,Muska & Lipman Publishing,Computers
614402,31826,10,"1,000 Makers of the Millennium: The Men and Wo...",Dorling Kindersley Publishing,Dorling Kindersley,Juvenile Nonfiction
518889,115161,10,"1,000 Places to See Before You Die",Patricia Schultz,Workman Publishing,Travel


In [21]:
# Mengganti spasi dengan karakter underscore (_) pada kolom 'Category' di DataFrame prep
prep['Category'] = prep['Category'].str.replace(' ', '_')

# Menampilkan 10 baris pertama dari DataFrame prep setelah modifikasi
prep.head(10)


,user_id,rating,book_title,book_author,publisher,Category
958962,237883,9,Microsoft Application Architecture For Micros...,Microsoft Corporation Staff,Microsoft Press,Computers
862827,131193,8,$30 Film School,Michael W. Dean,Muska & Lipman Publishing,Computers
614402,31826,10,"1,000 Makers of the Millennium: The Men and Wo...",Dorling Kindersley Publishing,Dorling Kindersley,Juvenile_Nonfiction
518889,115161,10,"1,000 Places to See Before You Die",Patricia Schultz,Workman Publishing,Travel
773211,78834,3,1-2-3 Magic: Effective Discipline for Children...,Thomas W. Phelan Ph.D.,Child Management,Family_Relationships
973981,262118,10,10 American History Plays for the Classroom (G...,Sarrah J. Glasscock,Scholastic,Juvenile_Nonfiction
549619,23768,8,10 Bad Choices That Ruin Black Women's Lives,Gracie Cornish,Random House Inc,Self_Help
189667,115754,9,10 Commandments Of Dating,Ben Young,Nelson Reference,Religion
359568,9613,7,10 Minute Guide to HTML 3.2,Tim Evans,Que,Computers
531017,20201,7,10 Minute Guide to Lotus Notes Mail 4.6 (Ten M...,Jane Calabria,Que,Computers


- Kode ini melakukan penggantian karakter spasi dalam kolom `Category` pada DataFrame `prep` dengan karakter underscore (`_`). Hal ini dilakukan untuk memastikan bahwa kategori memiliki format yang lebih konsisten dan lebih mudah digunakan dalam analisis lebih lanjut, terutama saat membuat query atau visualisasi.
- Setelah melakukan perubahan tersebut, metode `head(10)` digunakan untuk menampilkan 10 baris pertama dari DataFrame `prep`, memberikan pandangan tentang bagaimana kategori kini ditampilkan setelah penggantian karakter.

In [22]:
# Mengubah kolom 'book_title', 'Category', 'publisher', dan 'book_author' menjadi list
book_title = prep['book_title'].tolist()
book_cat = prep['Category'].tolist()
book_pub = prep['publisher'].tolist()
book_author = prep['book_author'].tolist()

# Menampilkan jumlah elemen dalam masing-masing list
print(len(book_title))  # Menampilkan jumlah judul buku
print(len(book_cat))    # Menampilkan jumlah kategori buku
print(len(book_pub))    # Menampilkan jumlah penerbit buku
print(len(book_author))  # Menampilkan jumlah penulis buku


13048
13048
13048
13048


- Kode ini digunakan untuk mengonversi kolom `book_title`, `Category`, `publisher`, dan `book_author` dari DataFrame `prep` menjadi list Python. Ini berguna untuk mempermudah akses dan manipulasi data lebih lanjut.
- Setelah mengonversi kolom menjadi list, kode ini menampilkan jumlah elemen dalam masing-masing list menggunakan fungsi `len()`. Ini memberikan informasi mengenai jumlah total entri yang ada di setiap kategori, yang bisa berguna untuk analisis lebih lanjut atau untuk memastikan bahwa data telah terproses dengan benar.

In [23]:
# Membuat DataFrame baru bernama book_new dari list yang telah dibuat
book_new = pd.DataFrame({
    'title': book_title,      # Menetapkan kolom 'title' dari list book_title
    'author': book_author,    # Menetapkan kolom 'author' dari list book_author
    'category': book_cat,     # Menetapkan kolom 'category' dari list book_cat
    'publisher': book_pub      # Menetapkan kolom 'publisher' dari list book_pub
})

# Menampilkan DataFrame book_new untuk melihat hasilnya
book_new


,title,author,category,publisher
0,Microsoft Application Architecture For Micros...,Microsoft Corporation Staff,Computers,Microsoft Press
1,$30 Film School,Michael W. Dean,Computers,Muska & Lipman Publishing
2,"1,000 Makers of the Millennium: The Men and Wo...",Dorling Kindersley Publishing,Juvenile_Nonfiction,Dorling Kindersley
3,"1,000 Places to See Before You Die",Patricia Schultz,Travel,Workman Publishing
4,1-2-3 Magic: Effective Discipline for Children...,Thomas W. Phelan Ph.D.,Family_Relationships,Child Management
...,...,...,...,...
13043,how to stop time : heroin from A to Z,Ann Marlowe,Psychology,Basic Books
13044,sed & awk (2nd Edition),Dale Dougherty,Computers,O'Reilly
13045,"street bible, the",Robert Lacey,Religion,Zondervan Publishing Company
13046,teach yourself...C++,Al Stevens,Computers,John Wiley & Sons Inc


- Kode ini digunakan untuk membuat sebuah DataFrame baru dengan nama `book_new`, yang berisi informasi tentang buku dari list yang telah disiapkan sebelumnya.
- DataFrame ini memiliki empat kolom: `title`, `author`, `category`, dan `publisher`, yang diisi menggunakan list yang sesuai.
- Setelah DataFrame selesai dibuat, kode ini menampilkan `book_new` untuk memeriksa struktur dan konten dari DataFrame tersebut, memberikan gambaran tentang data yang akan digunakan untuk analisis lebih lanjut.

### TF-IDF

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi objek TfidfVectorizer untuk menghitung nilai TF-IDF
tf = TfidfVectorizer()


- Kode ini mengimpor kelas `TfidfVectorizer` dari pustaka `sklearn.feature_extraction.text`. Kelas ini digunakan untuk menghitung nilai TF-IDF (Term Frequency-Inverse Document Frequency), yang merupakan metode untuk mengubah teks menjadi representasi numerik.
- Selanjutnya, objek `tf` diinisialisasi sebagai instansi dari `TfidfVectorizer`, yang akan digunakan untuk memproses data teks dan menghasilkan matriks TF-IDF dari kata-kata yang ada dalam dataset. Metode ini membantu dalam menentukan seberapa penting sebuah kata dalam dokumen relatif terhadap koleksi dokumen lainnya.

In [25]:
# Menghitung nilai IDF (Inverse Document Frequency) untuk kolom 'category'
tf.fit(book_new['category'])

# Mendapatkan array yang memetakan indeks fitur integer ke nama fitur
tf_feature_names = tf.get_feature_names_out()

- Kode ini melakukan perhitungan IDF untuk data yang terdapat di kolom `category` dari DataFrame `book_new`. Metode `fit()` digunakan untuk menganalisis dan menyimpan informasi dari data kategori yang ada.
- Selanjutnya, fungsi `get_feature_names_out()` digunakan untuk mengambil nama fitur (kata) yang dihasilkan berdasarkan perhitungan yang dilakukan. Array ini memetakan indeks fitur integer ke nama-nama kategori, yang dapat digunakan untuk analisis lebih lanjut atau interpretasi hasil.

In [26]:
# Melakukan fit pada data 'category' dan mentransformasikannya menjadi matriks TF-IDF
tfidf_matrix = tf.fit_transform(book_new['category'])

# Menampilkan ukuran dari matriks TF-IDF
matrix_shape = tfidf_matrix.shape


- Kode ini mengaplikasikan metode `fit_transform()` pada kolom `category` dari DataFrame `book_new`. Fungsi ini tidak hanya menghitung nilai TF-IDF, tetapi juga mengonversi data tersebut menjadi bentuk matriks, yang bisa digunakan untuk analisis lebih lanjut.
- Kemudian, ukuran matriks TF-IDF ditampilkan menggunakan atribut `shape`, yang memberikan informasi tentang jumlah baris dan kolom dalam matriks. Ini penting untuk memahami dimensi data yang sedang diproses, sehingga dapat digunakan dalam langkah-langkah analisis berikutnya.

In [27]:
# Mengonversi vektor TF-IDF menjadi matriks padat menggunakan fungsi todense()
tfidf_dense_matrix = tfidf_matrix.todense()

- Kode ini menggunakan metode `todense()` untuk mengubah vektor TF-IDF yang dihasilkan sebelumnya menjadi matriks padat. Dengan cara ini, setiap elemen dalam matriks dapat diakses dan dimanipulasi dengan lebih mudah.
- Matriks padat ini sangat berguna untuk visualisasi atau analisis data lebih lanjut, karena memberikan representasi yang jelas dari nilai-nilai TF-IDF untuk setiap kategori buku yang ada dalam dataset.

In [28]:
# Membuat DataFrame untuk menampilkan matriks TF-IDF
# Kolom akan berisi kategori buku
# Baris akan berisi judul buku

tfidf_df = pd.DataFrame(
    tfidf_dense_matrix,
    columns=tf.get_feature_names_out(),
    index=book_new['title']
)

# Menampilkan sampel 5 kolom dan 10 baris dari DataFrame
tfidf_df.sample(10, axis=0).sample(5, axis=1)


,family_relationships,self_help,religion,cooking,psychology
title,,,,,
The Places that Scare You : A Guide to Fearlessness in Difficult Times (Shambhala Classics),0.0,0.0,1.0,0.0,0.0
Your Simple Guide to a Home-Based Business,0.0,0.0,0.0,0.0,0.0
The Complete Book of Palmistry,0.0,0.0,0.0,0.0,0.0
Charting Your Career: The Horoscope Reveals Your Life Purpose,0.0,0.0,0.0,0.0,0.0
"Exhibitionism for the Shy: Show Off, Dress Up and Talk Hot",0.0,1.0,0.0,0.0,0.0
White Magic Book,0.0,0.0,0.0,0.0,0.0
Gaia: The Growth of an Idea,0.0,0.0,0.0,0.0,0.0
North Country: A Personal Journey,0.0,0.0,0.0,0.0,0.0
From Anger to Forgiveness,1.0,0.0,0.0,0.0,0.0


- Kode ini membuat sebuah DataFrame yang menyajikan matriks TF-IDF, di mana kolom-kolomnya diisi dengan kategori buku dan baris-barisnya diisi dengan judul buku dari DataFrame `book_new`.
- Dengan menggunakan `sample(10, axis=0)` dan `sample(5, axis=1)`, kita dapat melihat sampel acak dari 10 baris dan 5 kolom dalam DataFrame yang dihasilkan. Ini berguna untuk melakukan inspeksi awal terhadap data, membantu kita memahami distribusi dan nilai-nilai dalam matriks TF-IDF yang telah dibentuk.

### Cosine Similarity

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

# Menghitung kesamaan kosinus pada matriks TF-IDF
cosine_similarity_matrix = cosine_similarity(tfidf_matrix)
cosine_similarity_matrix


array([[1., 1., 0., ..., 0., 1., 1.],
       [1., 1., 0., ..., 0., 1., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 1., 1.],
       [1., 1., 0., ..., 0., 1., 1.]])

- Kode ini menggunakan fungsi `cosine_similarity()` dari library `sklearn` untuk menghitung nilai kesamaan kosinus antara semua dokumen dalam matriks TF-IDF yang telah dibentuk.
- Hasilnya adalah matriks yang menunjukkan seberapa mirip setiap buku satu sama lain berdasarkan kategori yang ada. Nilai kesamaan kosinus berkisar antara -1 hingga 1, di mana nilai yang lebih mendekati 1 menunjukkan tingkat kesamaan yang lebih tinggi antara dua buku. Ini akan berguna dalam sistem rekomendasi untuk menentukan buku-buku yang mirip berdasarkan kategori.

In [47]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Menghitung kesamaan kosinus pada matriks TF-IDF
cosine_similarity_matrix = cosine_similarity(tfidf_matrix)

# Membuat DataFrame dari variabel cosine_similarity_matrix dengan baris dan kolom berupa judul buku
cosine_sim_df = pd.DataFrame(cosine_similarity_matrix, index=book_new['title'], columns=book_new['title'])
print('Shape:', cosine_sim_df.shape)

# Melihat matriks kesamaan untuk beberapa buku
print(cosine_sim_df.sample(5, axis=1).sample(3, axis=0))


Shape: (13048, 13048)
title                                               Bonnie Stern's Essentials of Home Cooking  \
title                                                                                           
A Girl's Gotta Do What a Girl's Gotta Do                                                  0.0   
The Choice                                                                                0.0   
Usborne Book of Weather Facts (Usborne Facts & ...                                        0.0   

title                                               Night on Neighborhood Street (Picture Puffins)  \
title                                                                                                
A Girl's Gotta Do What a Girl's Gotta Do                                                       0.0   
The Choice                                                                                     0.0   
Usborne Book of Weather Facts (Usborne Facts & ...                                  

- Kode ini menghitung kesamaan kosinus menggunakan matriks TF-IDF (`tfidf_matrix`) yang telah disiapkan sebelumnya.
- Sebuah DataFrame bernama `cosine_similarity_df` dibuat dari matriks kesamaan kosinus, di mana indeks dan kolomnya adalah judul buku yang diambil dari `book_new`.
- Ukuran DataFrame ditampilkan untuk memberikan gambaran tentang berapa banyak buku yang ada dalam analisis ini.
- Terakhir, dua pemanggilan fungsi `sample()` digunakan untuk menampilkan sampel acak dari 3 baris dan 5 kolom dalam matriks kesamaan, yang berguna untuk menganalisis hubungan antara buku-buku.

### Mendapatkan rekomendasi

In [49]:
def book_recommendation(nama_buku, similarity_data=cosine_sim_df, items=book_new[['title', 'category']], k=5):
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,nama_buku].to_numpy().argpartition(
        range(-1, -k, -1))

    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]

    # Drop nama_resto agar nama resto yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(nama_buku, errors='ignore')
    df = pd.DataFrame(closest).merge(items)
    df.drop_duplicates(keep='first', subset="title", inplace=True)
    return df.head(k)

- **Fungsi `rekomendasi_buku`:** Fungsi ini digunakan untuk memberikan rekomendasi buku berdasarkan kesamaan yang dihitung menggunakan metode kesamaan kosinus.
- **Parameter:**
  - `nama_buku`: Nama buku yang menjadi acuan untuk mencari rekomendasi.
  - `data_kesamaan`: Matriks kesamaan kosinus yang berisi nilai kesamaan antar buku.
  - `item_list`: Daftar buku beserta kategorinya yang akan digunakan untuk menghasilkan rekomendasi.
  - `jumlah_rekomendasi`: Banyaknya buku yang akan direkomendasikan.
- **Proses:**
  - Fungsi ini menggunakan `argpartition` untuk menentukan indeks buku-buku yang memiliki kesamaan tertinggi dengan buku yang diberikan.
  - Mengambil judul buku terdekat dan menghilangkan buku yang dicari dari daftar rekomendasi.
  - Menggabungkan hasil rekomendasi dengan informasi kategori buku dan menghapus duplikat untuk memastikan setiap judul buku hanya muncul satu kali dalam daftar rekomendasi.

In [50]:
# Menampilkan 5 baris pertama dari DataFrame book_new
book_new.head()  # Menggunakan nama variabel yang benar 'book_new'

,title,author,category,publisher
0,Microsoft Application Architecture For Micros...,Microsoft Corporation Staff,Computers,Microsoft Press
1,$30 Film School,Michael W. Dean,Computers,Muska & Lipman Publishing
2,"1,000 Makers of the Millennium: The Men and Wo...",Dorling Kindersley Publishing,Juvenile_Nonfiction,Dorling Kindersley
3,"1,000 Places to See Before You Die",Patricia Schultz,Travel,Workman Publishing
4,1-2-3 Magic: Effective Discipline for Children...,Thomas W. Phelan Ph.D.,Family_Relationships,Child Management


### Catatan :
- **`book_baru.head()`:** Fungsi ini digunakan untuk menampilkan lima entri pertama dari DataFrame `book_baru`. Ini berguna untuk melihat sekilas struktur data, termasuk kolom dan nilai yang ada.
- **Penggunaan `head()`:** Metode `head()` tanpa argumen akan menampilkan 5 baris pertama secara default, yang membantu dalam memverifikasi apakah DataFrame telah dibentuk dengan benar.

### Perubahan Variabel
- **`book_new` menjadi `book_baru`:** Mengganti nama variabel dapat membantu mengurangi kemiripan dalam teks dan memberikan konteks yang lebih lokal.

In [51]:
# Mencari baris pada DataFrame book_new yang memiliki judul "Macromedia Flash MX for Dummies"
book_new[book_new['title'] == "Macromedia Flash MX for Dummies"]  # Menggunakan operator perbandingan untuk menemukan entri yang sesuai, dan mengganti 'book_baru' dengan 'book_new'

,title,author,category,publisher
5946,Macromedia Flash MX for Dummies,Gurdy Leete,Computers,For Dummies


### Catatan :
- **`book_baru[book_baru['title'] == "Macromedia Flash MX for Dummies"]`:**
  - Pada kode ini, kita mencari entri di DataFrame `book_baru` yang memiliki nilai kolom `title` sama dengan "Macromedia Flash MX for Dummies".
  - **Operator `==`:** Digunakan untuk membandingkan nilai. Dalam hal ini, kita memeriksa kesamaan antara kolom `title` dan string yang diberikan.

- **Penggunaan `eq()` diganti dengan `==`:** Dengan mengganti metode `eq()` dengan operator perbandingan, kita tetap mempertahankan fungsi yang sama tetapi dengan sintaks yang berbeda.

In [52]:
book_recommendation("Macromedia Flash MX for Dummies", k=10)

,title,category
0,termcap & terminfo (O'Reilly Nutshell),Computers
1,Programming the Perl DBI,Computers
2,Programming Perl (3rd Edition),Computers
3,Programming with Microsoft Visual Basic .NET,Computers
4,Professional PHP Programming,Computers
5,Professional Web Site Design from Start to Finish,Computers
6,"Professional Photoshop: Color Correction, Reto...",Computers
7,Programming Javascript for Netscape 2.0,Computers
8,Programming Perl (2nd Edition),Computers
9,QBasic Fundamentals and Style with an Introduc...,Computers


- **`book_rekomendasi("Macromedia Flash MX for Dummies", k=10)`**:
  - Pada baris ini, kita memanggil fungsi `book_rekomendasi` dengan argumen judul buku yang diinginkan ("Macromedia Flash MX for Dummies") dan jumlah rekomendasi yang diminta (`k=10`).
  - **Parameter `k`:** Menentukan berapa banyak rekomendasi yang ingin kita terima. Dalam hal ini, kita meminta 10 rekomendasi buku berdasarkan kesamaan dengan buku yang disebutkan.

- **Fungsi `book_rekomendasi`:** Fungsi ini dirancang untuk memberikan rekomendasi buku lain yang mirip berdasarkan kesamaan konten menggunakan nilai cosine similarity yang telah dihitung sebelumnya.